In [3]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import random
import pandas as pd
import xlsxwriter
from io import BytesIO
import requests

In [70]:
id_ = input('아이디를 입력하세요:   ')
password_ = input('비밀번호를 입력하세요:   ')
search = input('검색할 내용을 입력 하세요:   ')
search_count = int(input('몇건을 검색 할까요:   '))

아이디를 입력하세요:   
비밀번호를 입력하세요:   
검색할 내용을 입력 하세요:   
몇건을 검색 할까요:   1


In [71]:

chrome_path = 'C:\chromedriver\chromedriver.exe'
driver = webdriver.Chrome(chrome_path)
url = 'https://www.instagram.com/'
driver.get(url)
time.sleep(3)

## 한꺼번에 입력하거나 자주 실행하면 막힘.....ㅎㅎ
element = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input')
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').click()
for i in id_:
    element.send_keys(i)
    time.sleep(random.uniform(0.1,0.3))

element = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input')
driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').click()
for i in password_:
    element.send_keys(i)
    time.sleep(random.uniform(0.1,0.3))
time.sleep(3)

driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]').click()
time.sleep(2)

## 자동로그인 설정
driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/div/div/button').click()
time.sleep(2)

## 알림설정
driver.find_element_by_xpath('/html/body/div[5]/div/div/div/div[3]/button[2]').click()
time.sleep(2)

driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[1]/div').click()
element = driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input')

for i in search:
    element.send_keys(i)
    time.sleep(random.uniform(0.1,0.3))

time.sleep(2)
element.send_keys('\n')
time.sleep(3)
element.send_keys('\n')

time.sleep(6)
title_=[]
hash_tag = []
txt_ = []
img=[]

driver.find_element_by_class_name('_9AhH0').click()
time.sleep(3)

html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

try:
    title_text = soup.find('a',class_='O4GlU').get_text()
    title_.append(title_text)
except:
    pass

txt_text = soup.select_one('div.C4VMK>span').get_text()
txt_.append(txt_text)

review_soup= soup.select('a.xil3i')
a = ''
for i in review_soup:
    a += i.get_text()+' '
hash_tag.append(a)


In [10]:
print(hash_tag)
print(title_)
print(txt_)

['#키친187 #창원맛집 #창원카페 #창원파스타 #창원데이트 #창원 #창원여행 #창원스테이크 #창원분위기좋은곳 #동정동맛집 #북면맛집 ']
[]
['..창원에는 거제와는 또다른 매력❤️아니 마력이라고 해야하나? ㅎㅎ.연인 데이트, 가족나들이로 아주 좋은메뉴들로 구성 해놨습니다~^^주말에는 브레이크 타임도 없습니다👍👍.#키친187#창원맛집#창원카페#창원파스타#창원데이트#창원#창원여행#창원스테이크#창원분위기좋은곳#동정동맛집#북면맛집']


In [32]:
driver.find_element_by_class_name('_9AhH0').click()

C:\Users\LIMDON~1\AppData\Local\Temp/ipykernel_41172/1583476414.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_class_name('_9AhH0').click()


In [183]:
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

review_soup= soup.select_one('div.C4VMK>span').get_text()
review_soup

'진해 웅천 읍성에서 만난 옹성옹성이란성문앞에 설치되는 시설물(성벽)인데 그 형상이 항아리 모양을 하고 있어 붙여진 이름. 성문을 부수고 침입하려는 적군을 측면과 후방에서 공격할 수 있다. 포위된 상태를 만들어 전방위로 공격할 수 있어 아군에게 유리한 요새 시설이라고 개인적으로 추측옹성을 설명할려면 성곽위에서 내려다보며 촬영해야 이해하기가 쉽다 하지만 구성을 위한 촬영이다보니 들어가는 입구에서 촬영함#Nikon #nikonz6 #ALLaboutNIKON #Web04 #changwonCITY #Landscape #웅천읍성 #웅성 #읍성 #해자 #치와문루 #창원시티 #진해구 #창원여행'

In [192]:
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

title_text = soup.find('a',class_='O4GlU').get_text()
title_.append(title_text)

In [193]:
title_

['부산핫플레이스']

In [66]:
## 사진 url
a= 100
img_url_ = []
while True:
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    list_a = soup.find('article','KC1QD').find_all('img')
    for i in list_a:
        list_url = i.get('src')
        img_url_.append(list_url)
    img_url = set(img_url_)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(3)
    if len(img_url)>= a:
        img_url = list(img_url)[:50]
        break

In [126]:
for i in img_url:
    print(i)

https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/273225861_390229286196039_760451436023459337_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=103&_nc_ohc=W_7nO9mtr4IAX-mFj6I&edm=AMKDjl4BAAAA&ccb=7-4&ig_cache_key=Mjc2NzMzNzMwOTk3OTc3NDY1NQ%3D%3D.2-ccb7-4&oh=00_AT__CoKT3Ptf30xN379XyqePiPMkuW2YRbUp6abR4pIRew&oe=620F6EBF&_nc_sid=1fe099
https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/273523785_204533165179990_4681917586295743477_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=110&_nc_ohc=9-EL8zD86-wAX8gStmd&tn=wdaAbHRLG3Pkz4i7&edm=AMKDjl4BAAAA&ccb=7-4&ig_cache_key=Mjc2ODA4NTkzNDI2NTM3NDI0Mg%3D%3D.2-ccb7-4&oh=00_AT8d5SOvApNm7p5ZIO6bKg-15eW97UwSeCfgmmJXBMJJ0A&oe=6210657A&_nc_sid=1fe099
https://scontent-gmp1-1.cdninstagram.com/v/t51.2885-15/e35/273369752_1626943780976299_1135823876727927454_n.jpg?_nc_ht=scontent-gmp1-1.cdninstagram.com&_nc_cat=107&_nc_ohc=E5LVQGTpLSsAX9PEbq7&edm=AMKDjl4BAAAA&ccb=7-4&ig_cache_key=Mjc2Njk4MDYwOTk1NDY0MjgyNQ%3D%3D.2-ccb7-4&oh=00_A

 다음에 할일:
 사진 저장 하고 엑셀에 사진과 title,내용,hash_tag내용 입력